In [ ]:
import math
import random

In [ ]:
def distL2(x1,y1, x2,y2):
    xdiff = x2 - x1
    ydiff = y2 - y1
    return int(math.sqrt(xdiff*xdiff + ydiff*ydiff) + .5)

In [ ]:
def distL1(x1,y1, x2,y2):
    return int(abs(x2-x1) + abs(y2-y1)+.5)

In [ ]:
def calc_matriz(coord, dist):
    n = len(coord)
    D = {}
    for i in range(n-1):
        for j in range(i+1,n):
            (x1,y1) = coord[i]
            (x2,y2) = coord[j]
            D[i,j] = dist(x1,y1,x2,y2)
            D[j,i] = D[i,j]
    return n,D

In [ ]:
def read_tsplib(filename):
    f = open(filename, 'r');

    line = f.readline()
    while line.find("EDGE_WEIGHT_TYPE") == -1:
        line = f.readline()

    if line.find("EUC_2D") != -1:
        dist = distL2
    elif line.find("MAN_2D") != -1:
        dist = distL1
    else:
        print("cannot deal with non-euclidean or non-manhattan distances")
        raise Exception

    while line.find("NODE_COORD_SECTION") == -1:
        line = f.readline()

    xy_positions = []
    while 1:
        line = f.readline()
        if line.find("EOF") != -1: break
        (i,x,y) = line.split()

        x = float(x)
        y = float(y)
        xy_positions.append((x,y))

    n,D = calc_matriz(xy_positions, dist)
    return n, xy_positions, D

In [ ]:
def custo(tour):
    z = D[tour[-1], tour[0]]
    for i in range(1,len(tour)):
        z += D[tour[i], tour[i-1]]
    return z,

In [ ]:
def randtour(n):
    sol = list(range(n))
    random.shuffle(sol)
    return sol


In [ ]:
dataset = 'a280.tsp'
path = './'+dataset
print(path)

n, coord, D = read_tsplib(path)

./a280.tsp


In [ ]:
print("Solução aletória:")
tour = randtour(n)
z = custo(tour)
print(f'Rota: {tour}\ncusto: {z}')

Solução aletória:
Rota: [140, 15, 72, 240, 223, 59, 110, 272, 175, 203, 56, 264, 104, 42, 76, 125, 238, 234, 89, 14, 137, 249, 260, 27, 30, 220, 202, 158, 111, 71, 133, 96, 123, 65, 66, 198, 276, 224, 209, 2, 207, 100, 195, 246, 113, 8, 251, 232, 179, 257, 38, 106, 194, 248, 139, 41, 20, 11, 131, 227, 9, 190, 160, 47, 69, 226, 174, 43, 34, 13, 241, 228, 115, 225, 77, 7, 156, 92, 210, 144, 52, 189, 78, 142, 40, 54, 95, 180, 273, 120, 176, 256, 81, 18, 82, 90, 53, 214, 201, 145, 37, 171, 247, 124, 103, 250, 143, 269, 265, 167, 127, 200, 159, 235, 197, 132, 185, 219, 244, 162, 21, 64, 73, 266, 80, 23, 199, 153, 178, 166, 126, 5, 121, 149, 218, 208, 157, 192, 31, 245, 86, 22, 87, 147, 237, 74, 109, 61, 17, 165, 205, 215, 130, 136, 196, 262, 236, 243, 119, 134, 230, 233, 204, 39, 183, 184, 146, 270, 168, 163, 94, 79, 161, 101, 128, 57, 98, 93, 45, 211, 173, 33, 107, 129, 187, 254, 222, 19, 85, 83, 271, 12, 68, 274, 26, 28, 122, 105, 50, 84, 58, 70, 170, 116, 117, 206, 263, 25, 135, 63, 114,

In [ ]:
!pip install deap

In [ ]:
import random
import numpy
from deap import base
from deap import creator
from deap import algorithms
from deap import tools

In [ ]:
toolbox = base.Toolbox()

creator.create("Minimizacao", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.Minimizacao)

toolbox.register("aleatorios", random.sample, range(n), n)
toolbox.register("gene", tools.initIterate, creator.Individual, toolbox.aleatorios)
toolbox.register("population", tools.initRepeat, list, toolbox.gene)

toolbox.register("evaluate", custo)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)


In [ ]:
def statistics(individual):
  return individual.fitness.values

In [ ]:
vezes = 10

for n in range(vezes):

    populacao = toolbox.population(100)

    prob_cross = 0.8
    prob_mut = 0.2
    geracoes = 100


    stats = tools.Statistics(statistics)
    stats.register("média", numpy.mean)
    stats.register("desvio padrão", numpy.std)

    populacao, info = algorithms.eaSimple(
        populacao,
        toolbox,
        prob_cross,
        prob_mut,
        geracoes,
        stats,
        verbose=False
)

    melhores = tools.selBest(populacao, 1)

    for individuo in melhores:
        print(individuo)
        print("Melhor Solução:", individuo.fitness)
        print("--------------------------------------")
        estatisticas = stats.compile(populacao)
        print(estatisticas)

[112, 277, 3, 2, 243, 235, 217, 191, 221, 213, 116, 215, 212, 270, 260, 206, 14, 27, 25, 67, 154, 272, 64, 100, 178, 117, 79, 43, 183, 162, 42, 155, 266, 54, 74, 89, 76, 143, 261, 157, 207, 180, 120, 193, 233, 188, 4, 24, 133, 6, 119, 156, 171, 199, 130, 33, 19, 121, 21, 205, 226, 244, 248, 216, 247, 240, 144, 190, 146, 150, 23, 274, 220, 135, 149, 101, 105, 168, 177, 241, 225, 227, 264, 139, 256, 273, 8, 9, 127, 26, 22, 169, 96, 93, 158, 218, 254, 103, 161, 187, 166, 10, 137, 136, 175, 34, 124, 151, 167, 68, 56, 86, 85, 37, 31, 95, 106, 164, 189, 172, 71, 118, 122, 91, 109, 73, 77, 66, 49, 102, 94, 159, 60, 46, 50, 38, 32, 35, 185, 110, 173, 107, 41, 48, 81, 214, 223, 257, 198, 224, 234, 194, 104, 92, 69, 84, 83, 108, 271, 126, 29, 152, 259, 7, 1, 228, 20, 163, 195, 267, 18, 201, 279, 210, 255, 208, 269, 28, 30, 123, 44, 47, 52, 179, 249, 138, 111, 113, 219, 202, 239, 242, 237, 17, 238, 12, 141, 147, 204, 230, 0, 132, 184, 59, 88, 80, 87, 174, 265, 250, 252, 253, 176, 170, 200, 192, 1